# News Scraping

## Extract

In [222]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [223]:
urls = [
    "https://www.cnnindonesia.com/nasional/20251119230659-20-1297303/marak-video-amatir-erupsi-semeru-khofifah-ingatkan-warga-jaga-jarak",
    "https://www.cnnindonesia.com/nasional/20251118080935-12-1296568/korban-bullying-tewas-dan-hilangnya-rasa-aman-di-sekolah",
    "https://www.cnnindonesia.com/nasional/20251106101927-32-1292524/gimik-politik-sidang-mkd-di-balik-sanksi-sahroni-nafa-urbach-cs",
    "https://www.cnnindonesia.com/ekonomi/20251021062509-532-1286684/melihat-ekonomi-ri-setahun-prabowo-gibran-adakah-yang-perlu-dibenahi",
    "https://www.cnnindonesia.com/ekonomi/20251016061332-532-1285042/tepatkah-purbaya-mau-hapus-utang-rp1-juta-biar-warga-bisa-ajukan-kpr",
    "https://www.cnnindonesia.com/nasional/20251009111137-32-1282674/catatan-untuk-komite-otsus-urus-papua-bareng-bkp3-pimpinan-gibran",
    "https://www.cnnindonesia.com/nasional/20251119131113-12-1297058/refly-harun-walk-out-usai-roy-suryo-cs-ditolak-reformasi-polri-jimly",
    "https://www.cnnindonesia.com/nasional/20251119232202-12-1297306/kasus-korupsi-kuota-haji-kpk-sita-1-rumah-dan-sejumlah-kendaraan",
    "https://www.cnnindonesia.com/nasional/20251119140232-32-1297089/alasan-fraksi-pks-copot-mardani-ali-sera-dari-ketua-bksap-dpr",
    "https://www.cnnindonesia.com/nasional/20251119235105-20-1297307/sekitar-300-warga-dievakuasi-antisipasi-letusan-susulan-gunung-semeru",
    "https://www.cnnindonesia.com/ekonomi/20251119190500-92-1297252/bp-bumn-buka-suara-soal-isu-delisting-saham-waskita-imbas-merger",
    "https://www.cnnindonesia.com/nasional/20251119165436-20-1297192/kpu-solo-klarifikasi-berkas-jokowi-baru-setahun-sudah-dimusnahkan",
    "https://www.cnnindonesia.com/ekonomi/20251119174800-85-1297227/bos-pertamina-respons-keluhan-dpr-soal-bbm-di-jatim",
    "https://www.cnnindonesia.com/ekonomi/20251119194621-532-1297275/riset-pdb-ri-lewati-titik-terendah-siap-tumbuh-52-persen-di-2026",
    "https://www.cnnindonesia.com/internasional/20250925125524-134-1277599/presiden-indonesia-ini-minta-hak-veto-dk-pbb-ditinjau-ulang",
    "https://www.cnnindonesia.com/olahraga/20251119074004-142-1296919/performa-indonesia-u-23-menjanjikan-emas-sea-games-2025-masih-jauh",
    "https://www.cnnindonesia.com/olahraga/20251119133352-142-1297070/daftar-top-skor-kualifikasi-piala-dunia-2026-haaland-runcing",
    "https://www.cnnindonesia.com/olahraga/20251119191344-142-1297260/8-negara-peserta-piala-dunia-2026-yang-pernah-dihajar-timnas-indonesia",
    "https://www.cnnindonesia.com/olahraga/20251119134608-142-1297076/indra-isyaratkan-pemain-abroad-timnas-u-23-belum-pasti-ke-sea-games",
    "https://www.cnnindonesia.com/olahraga/20251121161024-142-1297995/perdana-menteri-malaysia-ultimatum-fam-ini-masalah-besar",
    "https://www.cnnindonesia.com/gaya-hidup/20251122164038-269-1298373/industri-wisata-jepang-masih-santuy-meski-diboikot-china",
    "https://www.cnnindonesia.com/nasional/20251122195352-20-1298416/gus-yahya-buka-suara-soal-isu-pemakzulan-ketum-pbnu",
    "https://www.cnnindonesia.com/nasional/20251122191926-20-1298412/koalisi-sipil-gugat-ke-mk-lapor-pbb-jika-kuhap-baru-diberlakukan",
    "https://www.cnnindonesia.com/ekonomi/20251121060653-92-1297746/melihat-risiko-jika-pppk-dijadikan-pns-besarkah?mtype=mpc.ctr.A-boxccxmpcxmp-modelA",
    "https://www.cnnindonesia.com/internasional/20251122153458-134-1298346/trump-ancam-hukum-mati-anggota-kongres-demokrat-yang-berkhianat",
    "https://www.cnnindonesia.com/nasional/20251122153851-12-1298349/rudy-tanoe-kembali-ajukan-praperadilan-gugat-status-tersangka-kpk",
    "https://www.cnnindonesia.com/nasional/20251122164531-25-1298375/bgn-beri-pelatihan-ribuan-penjamah-makanan-untuk-sppg-sulsel",
    "https://www.cnnindonesia.com/otomotif/20251122145725-603-1298339/byd-kuasai-54-persen-penjualan-mobil-listrik-indonesia",
    "https://www.cnnindonesia.com/internasional/20251122115513-134-1298265/sempat-sesumbar-pangkas-dana-new-york-trump-kini-siap-bantu-mamdani",
    "https://www.cnnindonesia.com/ekonomi/20251122150853-625-1298342/sedia-akses-andal-jtt-perkuat-konektivitas-wilayah-timur-trans-jawa",
    "https://www.cnnindonesia.com/ekonomi/20251122162821-532-1298371/mensos-sebut-blts-tahap-dua-cair-pekan-depan",
    "https://www.cnnindonesia.com/nasional/20251121164557-20-1298046/kawasan-semeru-hujan-deras-banjir-lahar-dingin-kembali-meluap",
    "https://www.cnnindonesia.com/hiburan/20251121130455-220-1297906/sinopsis-brick-mansions-bioskop-trans-tv-22-november-2025",
    "https://www.cnnindonesia.com/nasional/20251121172523-20-1298054/suami-istri-tewas-tertimpa-pohon-tumbang-di-sleman-yogyakarta",
    "https://www.cnnindonesia.com/nasional/20251122171347-32-1298383/koalisi-sipil-desak-prabowo-terbitkan-perppu-batalkan-kuhap-baru",
    "https://www.cnnindonesia.com/ekonomi/20251121190232-625-1298111/ptpn-1-raih-tiga-penghargaan-bergengsi",
    "https://www.cnnindonesia.com/nasional/20251121201119-12-1298144/kronologi-pembakaran-rumah-hakim-pn-medan-versi-polisi",
    "https://www.cnnindonesia.com/internasional/20251121165857-120-1298026/israel-masih-bombardir-brutal-gaza-sampai-tepi-barat-saat-gencatan",
    "https://www.cnnindonesia.com/ekonomi/20251122132711-625-1298300/pupuk-kujang-ikut-rawat-citarum-demi-kelangsungan-pasokan-air-produksi",
]

In [224]:
def scrape_cnn(url):
    headers = {
        "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
    )
    }
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")

    judul_tag = soup.select_one("h1")
    judul = judul_tag.get_text(strip=True) if judul_tag else "Tidak ditemukan"

    tanggal_tag = soup.select_one("div.text-cnn_grey.text-sm.mb-4")
    tanggal = tanggal_tag.get_text(strip=True) if tanggal_tag else "Tidak ditemukan"

    author_tag = soup.select_one("figcaption")
    author = author_tag.get_text(strip=True) if author_tag else "Tidak ditemukan"

    paragraf = soup.select("p")
    isi = "\n".join(p.get_text(strip=True) for p in paragraf) if paragraf else "Isi tidak ditemukan"

    return {
        "judul": judul,
        "tanggal": tanggal,
        "author" : author,
        "isi": isi
    }




urls = list(set(urls))

results = []

for url in urls:
    print("Scraping:", url)
    try:
        data = scrape_cnn(url)
        results.append(data)
    except Exception as e:
        results.append({
            "url": url,
            "error": str(e)
        })
    time.sleep(1)

df = pd.DataFrame(results)
df.to_csv('cnnindo.csv', index=False, encoding='utf-8-sig')

print("Done")

Scraping: https://www.cnnindonesia.com/ekonomi/20251119190500-92-1297252/bp-bumn-buka-suara-soal-isu-delisting-saham-waskita-imbas-merger
Scraping: https://www.cnnindonesia.com/ekonomi/20251119194621-532-1297275/riset-pdb-ri-lewati-titik-terendah-siap-tumbuh-52-persen-di-2026
Scraping: https://www.cnnindonesia.com/olahraga/20251119074004-142-1296919/performa-indonesia-u-23-menjanjikan-emas-sea-games-2025-masih-jauh
Scraping: https://www.cnnindonesia.com/hiburan/20251121130455-220-1297906/sinopsis-brick-mansions-bioskop-trans-tv-22-november-2025
Scraping: https://www.cnnindonesia.com/nasional/20251119131113-12-1297058/refly-harun-walk-out-usai-roy-suryo-cs-ditolak-reformasi-polri-jimly
Scraping: https://www.cnnindonesia.com/nasional/20251119140232-32-1297089/alasan-fraksi-pks-copot-mardani-ali-sera-dari-ketua-bksap-dpr
Scraping: https://www.cnnindonesia.com/nasional/20251119165436-20-1297192/kpu-solo-klarifikasi-berkas-jokowi-baru-setahun-sudah-dimusnahkan
Scraping: https://www.cnnindon

## Transfrom

### CNN Indonesia

In [225]:
def clean_author(text):
    if text is None or not isinstance(text, str):
        return None

    match = re.search(r'/([A-Za-z\.\s\-]+)\)', text)
    if match:
        author = match.group(1).strip()
        return author if author else None

    return None
def clean_content(text):
    if text is None or not isinstance(text, str):
        return None

    t = text
    t = re.sub(r'ADVERTISEMENT.*?CONTENT', '', t, flags=re.IGNORECASE)
    t = re.sub(r'\[.*?CNN.*?\]', '', t)
    t = re.sub(r'\[.*?\]', '', t)
    t = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', t)
    t = re.sub(r'([a-z])([A-Z])', r'\1 \2', t)
    t = re.sub(r'\s+', ' ', t)
    t = re.sub(r'\s+([.,!?)])', r'\1', t)
    t = re.sub(r'([.,!?])([A-Za-z])', r'\1 \2', t)
    t = t.encode('ascii', 'ignore').decode('ascii')
    t = t.strip()
    return t

def clean_content(text):
    if text is None or not isinstance(text, str):
        return None

    t = text

    t = re.sub(
        r'ADVER.*?CONTINUE.*?CONTENT',
        '',
        t,
        flags=re.IGNORECASE | re.DOTALL
    )

    t = re.sub(r'\bADVERTISEMENT\b', '', t, flags=re.IGNORECASE)

    t = re.sub(
        r'SCROLL\s+TO\s+CONTINUE.*?CONTENT',
        '',
        t,
        flags=re.IGNORECASE
    )
    t = re.sub(r'\[.*?CNN.*?\]', '', t)
    t = re.sub(r'\[.*?\]', '', t)

    t = re.sub(r'\s+', ' ', t).strip()

    return t


In [226]:
df_cnn = pd.read_csv('cnnindo.csv')

In [227]:
df_cnn['author'] = df_cnn['author'].apply(clean_author)
df_cnn['isi'] = df_cnn['isi'].apply(clean_content)

In [228]:
df_cnn

,judul,tanggal,author,isi
0,BP BUMN Buka Suara Soal Isu Delisting Saham Wa...,"Rabu, 19 Nov 2025 20:55 WIB",Muhammad Falah Nafis,Badan Pengaturan Badan Usaha Milik Negara (BP ...
1,"Riset: PDB RI Lewati Titik Terendah, Siap Tumb...","Rabu, 19 Nov 2025 19:54 WIB",Adhi Wicaksono,RisetMacquarie CapitalIndonesia menyebutekonom...
2,"Performa Indonesia U-23 Menjanjikan, Emas SEA ...","Rabu, 19 Nov 2025 22:00 WIB",Muhammad Adimaja,Persiapan dan performaTimnas Indonesia U-23men...
3,"Sinopsis Brick Mansions, Bioskop Trans TV 22 N...",Tidak ditemukan,Brick Mansions Productions via IMDb,"Bioskop Trans TVmalam ini, Sabtu 22 November 2..."
4,Refly Harun Walk Out Usai Roy Suryo Cs Ditolak...,"Rabu, 19 Nov 2025 13:39 WIB",Khaira Ummah,Pakar Hukum Tata NegaraRefly Harunmemutuskanwa...
5,Alasan Fraksi PKS Copot Mardani Ali Sera dari ...,"Rabu, 19 Nov 2025 14:45 WIB",Hesti Rika,Ketua Fraksi PKS Abdul Kharis Almasyaharimengu...
6,KPU Solo Klarifikasi Berkas Jokowi Baru Setahu...,"Rabu, 19 Nov 2025 20:31 WIB",Rosyid,"Komisi Pemilihan Umum (KPU)Kota Solo, Jawa Ten..."
7,Pupuk Kujang Ikut Rawat Citarum demi Kelangsun...,"Sabtu, 22 Nov 2025 14:07 WIB",Adhi Wicaksono,"PT Pupuk Kujang, anak usaha PT Pupuk Indonesia..."
8,"Melihat Ekonomi RI Setahun Prabowo-Gibran, Ada...","Selasa, 21 Okt 2025 07:22 WIB",Ajeng Dinar Ulfiana,Setahun sudah pemerintahanPrabowo Subianto-Gib...
9,"Catatan untuk Komite Otsus, Urus Papua Bareng ...","Kamis, 09 Okt 2025 11:38 WIB",Ari Saputra,Pemerintah membentukKomite Eksekutif Percepata...


### BBC News

In [242]:
df_bbc = pd.read_excel('Scraping_BBC_result.xlsx')
df_bbc = df_bbc.head(10)
df_bbc

,Judul,Isi Lengkap Berita,Tanggal Publikasi,Nama Penulis
0,Indonesia names former dictator Suharto a 'nat...,Indonesia has named former president Suharto a...,2025-11-10 06:02:00,Jonathan Head; Koh Ewe
1,Lindsay Sandiford flown home after 12 years on...,A British grandmother who spent 12 years on de...,2025-11-07 03:57:00,Kelly Ng
2,IOC to tell members not to host events in Indo...,International sport federations will be told n...,2025-10-22 15:00:00,BBC Sport
3,British grandmother jailed in Indonesia for dr...,A British grandmother who has spent more than ...,2025-10-21 14:58:00,Dawn Limbu
4,Indonesia food poisonings and protests test Pr...,When Prabowo Subianto campaigned to become Ind...,2025-10-19 22:54:00,Suranjana Tewari
5,World Artistic Gymnastics Championships: Israe...,The Israeli Gymnastics Federation (IGF) says a...,2025-10-15 14:41:00,BBC Sport
6,World Artistic Gymnastics Championships: Israe...,"Israeli gymnasts, including defending world fl...",2025-10-14 13:45:00,BBC Sport
7,Indonesian MPs get extra allowance weeks after...,Weeks after widespread protests swept Indonesi...,2025-10-14 04:44:00,Koh Ewe; Astudestra Ajengrastri
8,Death toll from Indonesia school collapse rise...,The death toll from the collapse of a school i...,2025-10-06 06:26:00,Kelly Ng
9,Marc Marquez: MotoGP champion suffers collarbo...,MotoGP world champion Marc Marquez says he fee...,2025-10-05 11:03:00,BBC Sport


In [243]:
df_bbc = df_bbc.rename(columns={
    'Judul' : 'judul',
    'Tanggal Publikasi' : 'tanggal',
    'Nama Penulis' : 'author',
    'Isi Lengkap Berita' : 'isi',
})

In [244]:
kolom =  ['judul', 'tanggal', 'author','isi']
df_bbc = df_bbc[kolom]
df_bbc['tanggal'] = df_bbc['tanggal'].astype(str)
df_bbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   judul    10 non-null     object
 1   tanggal  10 non-null     object
 2   author   10 non-null     object
 3   isi      10 non-null     object
dtypes: object(4)
memory usage: 452.0+ bytes


In [245]:
df_bbc.to_csv('bbc.csv')

### CNBC Indonesia

In [246]:
df_cnbc1 = pd.read_csv('cnbc_indo_revised.csv')
df_cnbc2 = pd.read_csv('cnbc_indo_nirot.csv')

In [234]:
df_cnbc1.head()

,url,judul,author,tanggal,isi_konten
0,https://www.cnbcindonesia.com/news/20251120155...,"Anggaran MBG Baru Cair 58%, Sisa Rp 30 T di Ak...",Zahwa Madjid,20 November 2025 18:40,"Jakarta, CNBC Indonesia- Kementerian Keuangan ..."
1,https://www.cnbcindonesia.com/news/20251119160...,"MBG Jadi Sorotan di Jepang, Bahas Penggunaan B...",Khoirul Anam,19 November 2025 16:29,"Jakarta, CNBC Indonesia- Program Makan Bergizi..."
2,https://www.cnbcindonesia.com/news/20251120131...,Susu Ikan Jadi Menu MBG? Begini Jawaban KKP,Chandra Dwi Pranata,20 November 2025 14:25,"Jakarta, CNBC Indonesia- Wacana susu ikan akan..."
3,https://www.cnbcindonesia.com/news/20251114135...,"Makanan Sisa MBG Tambah Masalah Sampah, Pemkot...",Chandra Dwi Pranata,14 November 2025 17:35,"Cimahi, CNBC Indonesia -Program Makan Bergizi ..."
4,https://www.cnbcindonesia.com/news/20251117114...,"Bawa Banyak Manfaat, Begini Potret Kegembiraan...",Teti Purwanti,17 November 2025 11:51,"Jakarta,CNBC Indonesia- Suasana gembira penuh ..."


In [247]:
df_cnbc = pd.concat([df_cnbc1, df_cnbc2], ignore_index=True)
df_cnbc = df_cnbc.drop(columns=['url'])

In [248]:
df_cnbc = df_cnbc.rename(columns={
    'isi_konten' : 'isi'
})

In [249]:
df_cnbc.head()

,judul,author,tanggal,isi
0,"Anggaran MBG Baru Cair 58%, Sisa Rp 30 T di Ak...",Zahwa Madjid,20 November 2025 18:40,"Jakarta, CNBC Indonesia- Kementerian Keuangan ..."
1,"MBG Jadi Sorotan di Jepang, Bahas Penggunaan B...",Khoirul Anam,19 November 2025 16:29,"Jakarta, CNBC Indonesia- Program Makan Bergizi..."
2,Susu Ikan Jadi Menu MBG? Begini Jawaban KKP,Chandra Dwi Pranata,20 November 2025 14:25,"Jakarta, CNBC Indonesia- Wacana susu ikan akan..."
3,"Makanan Sisa MBG Tambah Masalah Sampah, Pemkot...",Chandra Dwi Pranata,14 November 2025 17:35,"Cimahi, CNBC Indonesia -Program Makan Bergizi ..."
4,"Bawa Banyak Manfaat, Begini Potret Kegembiraan...",Teti Purwanti,17 November 2025 11:51,"Jakarta,CNBC Indonesia- Suasana gembira penuh ..."


In [250]:
df_cnbc = df_cnbc[kolom]

## Merge

In [251]:
df_all = pd.concat([df_cnn, df_bbc, df_cnbc], ignore_index=True)

In [252]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   judul    70 non-null     object
 1   tanggal  70 non-null     object
 2   author   70 non-null     object
 3   isi      70 non-null     object
dtypes: object(4)
memory usage: 2.3+ KB


In [253]:
df_all

,judul,tanggal,author,isi
0,BP BUMN Buka Suara Soal Isu Delisting Saham Wa...,"Rabu, 19 Nov 2025 20:55 WIB",Muhammad Falah Nafis,Badan Pengaturan Badan Usaha Milik Negara (BP ...
1,"Riset: PDB RI Lewati Titik Terendah, Siap Tumb...","Rabu, 19 Nov 2025 19:54 WIB",Adhi Wicaksono,RisetMacquarie CapitalIndonesia menyebutekonom...
2,"Performa Indonesia U-23 Menjanjikan, Emas SEA ...","Rabu, 19 Nov 2025 22:00 WIB",Muhammad Adimaja,Persiapan dan performaTimnas Indonesia U-23men...
3,"Sinopsis Brick Mansions, Bioskop Trans TV 22 N...",Tidak ditemukan,Brick Mansions Productions via IMDb,"Bioskop Trans TVmalam ini, Sabtu 22 November 2..."
4,Refly Harun Walk Out Usai Roy Suryo Cs Ditolak...,"Rabu, 19 Nov 2025 13:39 WIB",Khaira Ummah,Pakar Hukum Tata NegaraRefly Harunmemutuskanwa...
...,...,...,...,...
65,Nasib 3 Astronaut China Terdampar di Antariksa...,20 November 2025 19:40,Redaksi,"Jakarta, CNBC Indonesia -Kejadian astronaut ya..."
66,Startup Tak Terkenal Mendadak Jadi Sorotan Gar...,20 November 2025 19:05,Thea Arbar,"Jakarta, CNBC Indonesia -xAI, startup kecerdas..."
67,"Kecanduan Scrolling Video Pendek Sudah Parah, ...",20 November 2025 17:15,Novina Putri Bestari,"Jakarta, CNBC Indonesia -Layanan video pendek ..."
68,Peneliti Temukan Jadwal Kiamat Dipercepat di B...,21 November 2025 06:38,Novina Putri Bestari,"Jakarta, CNBC Indonesia- Pemanasan global terj..."


In [254]:
df_all.to_excel('Kelompok 67.xlsx')